In [1]:
import json
import ast
from tqdm import tqdm
from vllm import LLM
from tqdm import tqdm
from vllm.sampling_params import SamplingParams

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

INFO 05-31 21:42:33 [__init__.py:239] Automatically detected platform cuda.


In [2]:
with open('/home/than/DeepLearning/CheXzero/data/mimic_pseudoans_cardiomegaly_0.1.json', 'r') as f:
    concepts = json.load(f).keys()

In [3]:
concepts = list(concepts)
len_concepts = len(concepts)
concepts[0]
print(f"Number of concepts: {len_concepts}")

'stable left basilar opacity, likely edema'

Number of concepts: 36599


### Concept Sets

In [4]:
with open('/home/than/DeepLearning/CheXzero/data/mimic_concepts.json', 'r') as f:
    results = json.load(f)

concepts = []
for result in tqdm(results):
    try:
        re_dict = ast.literal_eval(result['model_output'])
        concepts.extend(re_dict['observations'])
    except:
        # print(result['model_output'])
        continue
concepts = [concept.lower() for concept in concepts]
concepts = list(set(concepts))
length = len(concepts)
print(f"Total number of unique concepts: {length}")

100%|██████████| 377110/377110 [00:04<00:00, 83019.50it/s]


Total number of unique concepts: 368294


### Prepare LLM

In [3]:
model_name = "microsoft/phi-4" # "mistralai/Ministral-8B-Instruct-2410"
sampling_params = SamplingParams(max_tokens=8192, temperature=0, top_k=-1)
llm = LLM(model=model_name, 
          trust_remote_code=True,
          # tokenizer_mode="mistral", 
          # config_format="mistral", 
          # load_format="mistral"
          )

INFO 05-23 21:25:27 [config.py:689] This model supports multiple tasks: {'reward', 'generate', 'embed', 'classify', 'score'}. Defaulting to 'generate'.
INFO 05-23 21:25:27 [config.py:1901] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 05-23 21:25:28 [core.py:61] Initializing a V1 LLM engine (v0.8.4) with config: model='microsoft/phi-4', speculative_config=None, tokenizer='microsoft/phi-4', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=16384, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_backend=None), observability_config=ObservabilityConfig(show_hidden_metrics=False, otlp_traces_endpoint=None, collect_model_forwar

model-00005-of-00006.safetensors:   0%|          | 0.00/4.77G [00:00<?, ?B/s]

model-00004-of-00006.safetensors:   0%|          | 0.00/4.77G [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00006-of-00006.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

...

model-00003-of-00006.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

INFO 05-23 21:29:48 [weight_utils.py:281] Time spent downloading weights for microsoft/phi-4: 258.312272 seconds


model.safetensors.index.json:   0%|          | 0.00/20.4k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/6 [00:00<?, ?it/s]


INFO 05-23 21:29:53 [loader.py:458] Loading weights took 4.91 seconds
INFO 05-23 21:29:53 [gpu_model_runner.py:1291] Model loading took 27.3876 GiB and 263.911935 seconds
INFO 05-23 21:29:59 [backends.py:416] Using cache directory: /home/than/.cache/vllm/torch_compile_cache/90df98595b/rank_0_0 for vLLM's torch.compile
INFO 05-23 21:29:59 [backends.py:426] Dynamo bytecode transform time: 5.95 s
INFO 05-23 21:30:01 [backends.py:132] Cache the graph of shape None for later use
INFO 05-23 21:30:22 [backends.py:144] Compiling a graph for general shape takes 22.21 s
INFO 05-23 21:30:38 [monitor.py:33] torch.compile takes 28.16 s in total
INFO 05-23 21:30:39 [kv_cache_utils.py:634] GPU KV cache size: 71,248 tokens
INFO 05-23 21:30:39 [kv_cache_utils.py:637] Maximum concurrency for 16,384 tokens per request: 4.35x
INFO 05-23 21:31:09 [gpu_model_runner.py:1626] Graph capturing finished in 29 secs, took 1.99 GiB
INFO 05-23 21:31:09 [core.py:163] init engine (profile, create kv cache, warmup mode

In [8]:
pclass = "pneumonia"

prompt = f"""
You are a helpful assistant. Please respond in valid JSON only.
Question: Think as a radiologist and answer yes /no / depends for whether the following concepts are clinically relevant for diagnosing '{pclass}': {concepts[:15]}
Output format: <concept>: <answer>.
"""
print(prompt)


You are a helpful assistant. Please respond in valid JSON only.
Question: Think as a radiologist and answer yes /no / depends for whether the following concepts are clinically relevant for diagnosing 'pneumonia': ['Right upper lobe', 'Bilateral pleural effusions larger on the right side are stable', 'Near complete opacification of the right lung field', 'Air-filled loops of bowel under the hemidiaphragms bilaterally', 'Dobhoff tube terminating within the stomach', 'A right PICC tip is in the low SVC', 'Platelike atelectasis at the left lung bases persists', 'Trachea deviated rightward secondary to enlarged thyroid', 'Areas of platelike atelectasis at both lung bases and left hilus', 'Small amount of right subcutaneous emphysema', 'Right PICC line placement is unchanged', 'Mildly displaced fracture of the lateral right third rib', 'AP chest compared to ___:04 a.m.', 'Posttraumatic changes consistent with diaphragmatic rupture', 'Persistent right lower lobe peripheral airspace opacities

In [9]:
messages = [
    {
        "role": "user",
        "content": prompt
    },
]

outputs = llm.chat(messages, 
                sampling_params=sampling_params)

result = {
    "id": 0,
    "model_output": outputs[0].outputs[0].text,
}

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

In [10]:
print(result['model_output'])
# print(concepts[:5])
# json.loads(result['model_output'])

```json
{
  "Right upper lobe": "depends",
  "Bilateral pleural effusions larger on the right side are stable": "depends",
  "Near complete opacification of the right lung field": "yes",
  "Air-filled loops of bowel under the hemidiaphragms bilaterally": "no",
  "Dobhoff tube terminating within the stomach": "no",
  "A right PICC tip is in the low SVC": "no",
  "Platelike atelectasis at the left lung bases persists": "depends",
  "Trachea deviated rightward secondary to enlarged thyroid": "no",
  "Areas of platelike atelectasis at both lung bases and left hilus": "depends",
  "Small amount of right subcutaneous emphysema": "depends",
  "Right PICC line placement is unchanged": "no",
  "Mildly displaced fracture of the lateral right third rib": "no",
  "AP chest compared to ___:04 a.m.": "no",
  "Posttraumatic changes consistent with diaphragmatic rupture": "depends",
  "Persistent right lower lobe peripheral airspace opacities": "yes"
}
```


### Try ChatGPT

In [ ]:
from openai import AzureOpenAI

client = AzureOpenAI(
  azure_endpoint = "", 
  api_key="", 
  api_version="",
)

In [6]:
pclass = "cardiomegaly"

prompt = f"""
You are a helpful assistant. Please respond in valid JSON only.
Question: Answer yes/no/depends for whether the following concepts are clinically relevant for diagnosing '{pclass}': {concepts[:100]}
Output format: <concept>: <answer>.
"""
print(prompt)


You are a helpful assistant. Please respond in valid JSON only.
Question: Answer yes/no/depends for whether the following concepts are clinically relevant for diagnosing 'cardiomegaly': ['picc line needs to be advanced by approximately 8 cm', 'upper enteric drainage tube ends in a non-distended stomach', 'small pleural effusions, larger on the right', 'consolidation in the left lower lung continues to improve', 'et tube tip at 5.5 cm above the carina', 'slight interval improvement in aeration at the left lung base', 'possible left goiter or cervical adenopathy', 'possible underlying emphysema', 'substantial interval increase in ground-glass opacities', 'vascular stents in both brachiocephalic veins', 'evidence of cervical fusion device', 'healed lateral left rib fractures', 'additional support lines and tubes in appropriate position', 'substantial pleural effusion accompanied by areas of relatively extensive parenchymal collapse', 'no consolidation, pneumothorax or pleural effusion', 

In [7]:
response = client.chat.completions.create(
            model="gpt-4.1", 
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt},
            ],
            temperature=0.0,
        ).choices[0].message.content

In [8]:
print(response)
json.loads(response)

{
  "picc line needs to be advanced by approximately 8 cm": "no",
  "upper enteric drainage tube ends in a non-distended stomach": "no",
  "small pleural effusions, larger on the right": "no",
  "consolidation in the left lower lung continues to improve": "no",
  "et tube tip at 5.5 cm above the carina": "no",
  "slight interval improvement in aeration at the left lung base": "no",
  "possible left goiter or cervical adenopathy": "no",
  "possible underlying emphysema": "no",
  "substantial interval increase in ground-glass opacities": "no",
  "vascular stents in both brachiocephalic veins": "no",
  "evidence of cervical fusion device": "no",
  "healed lateral left rib fractures": "no",
  "additional support lines and tubes in appropriate position": "no",
  "substantial pleural effusion accompanied by areas of relatively extensive parenchymal collapse": "no",
  "no consolidation, pneumothorax or pleural effusion": "no",
  "associated decrease in bilateral pleural effusions": "no",
  "r

{'picc line needs to be advanced by approximately 8 cm': 'no',
 'upper enteric drainage tube ends in a non-distended stomach': 'no',
 'small pleural effusions, larger on the right': 'no',
 'consolidation in the left lower lung continues to improve': 'no',
 'et tube tip at 5.5 cm above the carina': 'no',
 'slight interval improvement in aeration at the left lung base': 'no',
 'possible left goiter or cervical adenopathy': 'no',
 'possible underlying emphysema': 'no',
 'substantial interval increase in ground-glass opacities': 'no',
 'vascular stents in both brachiocephalic veins': 'no',
 'evidence of cervical fusion device': 'no',
 'healed lateral left rib fractures': 'no',
 'additional support lines and tubes in appropriate position': 'no',
 'substantial pleural effusion accompanied by areas of relatively extensive parenchymal collapse': 'no',
 'no consolidation, pneumothorax or pleural effusion': 'no',
 'associated decrease in bilateral pleural effusions': 'no',
 'right lower lobe pne

In [16]:
with open('/home/than/DeepLearning/CheXzero/data/mimic_pseudoans_cardiomegaly_0.1.json', 'r') as f:
    mimic_pseudoans = json.load(f)
mimic_pseudoans

[['obscuration of the right hemidiaphragm posteriorly', 'no'],
 ['multiple small surgical clips in the right anterior chest wall', 'no'],
 ['left internal jugular catheter ends in the lower svc', 'no'],
 ['air inclusions in the right lateral chest wall', 'no'],
 ['air-filled loops of nondilated small bowel are present', 'no'],
 ['has not changed over the past week', 'depends'],
 ['small tent-like opacity at the right cardiophrenic border may represent focal atelectasis',
  'no'],
 ['large right lower lobe atelectasis is unchanged', 'no'],
 ['atelectasis and possible small effusion', 'no'],
 ['placement of a dobbhoff catheter that extends to the distal stomach', 'no'],
 ['left subclavian central venous catheter terminates at the upper portion of the svc',
  'no'],
 ['moderate bilateral pleural effusions, larger on the left side, have decreased in amount',
  'no'],
 ['right jugular vascular line ends at the origin of the svc', 'no'],
 ['lungs are hyperinflated consistent with underlying 